In [74]:
import os
import pickle
import numpy as np

# # import pickle file 
with open('../scenario-properties-controller.pkl', 'rb') as f:
    baseline = pickle.load(f)

# with open('../scenario-properties-controller_orig.pkl', 'rb') as f:
#     baseline = pickle.load(f)

In [75]:
import sympy as sp

# Equations
baseline.equations

# x0 initial population
baseline.x0

# baseline.
baseline.all_symbolic_vars

# solver
equations_flattened = [baseline.equations[i, j] for j in range(baseline.equations.cols) for i in range(baseline.equations.rows)]

full_lambda = sp.Matrix(sp.symbols([f'lambda_{i+1}' for i in range(baseline.n_shells)]))
full_lambda_flattened = [full_lambda[i, j] for j in range(full_lambda.cols) for i in range(full_lambda.rows)]

for i1 in range(baseline.n_shells):
    equations_flattened[i1] = equations_flattened[i1]+full_lambda_flattened[i1]

baseline.all_symbolic_vars = baseline.all_symbolic_vars + full_lambda_flattened

equations = [sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy') for eq in equations_flattened]

In [119]:
# equations_flattened
# baseline.all_symbolic_vars

# x0 = np.ones(len(baseline.all_symbolic_vars)) * 2
# N_shell = baseline.n_shells
# S_all = x0[:N_shell]
# -np.sum(np.log(S_all)**2) 

# c_eq = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*x0) for eq in equations_flattened])
# c_eq

array([ 1.74999988e+00,  1.74999988e+00,  1.74999989e+00,  1.74999989e+00,
        1.74999989e+00,  1.74999989e+00,  1.74999989e+00,  1.74999989e+00,
        1.74999990e+00,  1.74999990e+00,  1.74999990e+00,  1.74999990e+00,
        1.74999990e+00,  1.74999990e+00, -2.87792668e+03, -6.75582916e+02,
       -2.05740828e+02, -8.15955923e+01, -3.02434543e+01, -1.25980309e+01,
       -5.38518636e+00, -2.46889540e+00, -1.04416779e+00, -5.22228315e-01,
       -2.24074147e-01, -1.27757952e-01, -5.55442465e-02, -1.14202611e-01,
       -7.32832620e+02, -1.72010391e+02, -5.23660924e+01, -2.07527732e+01,
       -7.67606418e+00, -3.18269897e+00, -1.34596859e+00, -6.03346361e-01,
       -2.40547550e-01, -1.07641686e-01, -3.17223561e-02, -7.20020925e-03,
        1.11842919e-02, -3.75723300e-03])

In [142]:
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from scipy.integrate import solve_ivp

def PrfAll(x, baseline):
  """
  Python version of the PrfAll function.

  Args:
    x: Input array, where the first N_shell elements represent S_all.
    baseline: Dictionary containing data, including 'N_shell'.

  Returns:
    f: Objective function.
  """

  N_shell = baseline.n_shells
  S_all = x[:N_shell]  # Extract S_all from x

  # HIGH CAPACITY
  f = -np.sum(np.log(S_all)**2) 
  # f = -np.sum((S_all)) 

  # Uncomment for NOT HIGH CAPACITY scenarios (as needed)
  # f = - np.sum(np.log(S_all[5:8])) # shells at 500-600 km (Python index starts at 0)
  # f = - np.sum(S_all[6]) # shell at 550 km (Python index starts at 0)

  return f

def PrcAll(x, baseline):
    """
    Evaluates all constraints at once.

    Args:
      x: The optimization variables.
      baseline: Your baseline object containing symbolic variables.

    Returns:
      An array containing the results of all constraint function evaluations.
    """

    c_eq = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*x) for eq in equations_flattened])
    
    # c_ineq 
    # y_fail_l(k) = -(lambda * Dt * (1 - failure_rate_L) - S);
    # y_fail_u(k) = lambda * Dt * (1 - failure_rate_U) - S; 

    return c_eq

## Initial guess
x0 = np.ones(len(baseline.all_symbolic_vars)) * 2

## Bounds
lb = np.ones(len(baseline.all_symbolic_vars)) * 0
# lb = -np.inf
ub = np.inf

## Objective function
objective = PrfAll

## Constraints for SLSQP
# con1 = {'type': 'eq', 'fun': lambda x: PrcAll(x, baseline)}
# # con2 = {'type': 'ineq', 'fun': constraint2}
# nonlcon = con1
# opt = {'ftol': 1e-12, 'eps': 1e-18, 'disp': True, 'maxiter': 5e5} 

## Constraints for trust-constr: lb <= fun(x) <= ub
# nonlcon = NonlinearConstraint(lambda x: PrcAll(x, baseline), 0, 0)
nonlcon = NonlinearConstraint(lambda x: PrcAll(x, baseline), 0, 1e-3)

## Perform the optimization
result = minimize(objective, x0, args=(baseline), method='SLSQP', # SLSQP, trust-constr, 
                  jac=None, hess=None, hessp=None, 
                  bounds=Bounds(lb=lb, ub=ub, keep_feasible=False), 
                  constraints=nonlcon, 
                  tol=1e1,
                  options=None)

## Print the results
print("Optimal found at:", result.x)
print("Function value at optimal:", -result.fun)
print("Equality constraints:", np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*result.x) for eq in equations_flattened]))
result

Optimal found at: [3.02550363e+00 2.96194522e+00 2.94682393e+00 2.94892554e+00
 2.95319216e+00 2.96544162e+00 2.99003989e+00 3.04080579e+00
 3.13188717e+00 3.25814157e+00 3.34526650e+00 3.27351903e+00
 2.91759717e+00 3.07010333e+00 0.00000000e+00 1.16087397e-06
 6.15408600e-06 2.21504242e-05 7.24879700e-05 2.05610011e-04
 5.40733437e-04 1.31595408e-03 3.07113618e-03 6.19762937e-03
 1.15551982e-02 1.64089634e-02 1.89709225e-02 1.13548886e-02
 1.07819292e-03 3.84422638e-03 1.06351745e-02 2.47901908e-02
 5.76876060e-02 1.22357566e-01 2.48371109e-01 4.77871410e-01
 9.00294211e-01 1.50566636e+00 2.39496692e+00 3.08044950e+00
 3.47335270e+00 2.67357477e+00 3.79188012e-01 3.71243266e-01
 3.69353042e-01 3.69615788e-01 3.70149070e-01 3.71680298e-01
 3.74755043e-01 3.81100830e-01 3.92485988e-01 4.08267810e-01
 4.19158455e-01 4.10190067e-01 3.65699829e-01 3.84763051e-01]
Function value at optimal: 17.50058179116801


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -17.50058179116801
       x: [ 3.026e+00  2.962e+00 ...  3.657e-01  3.848e-01]
     nit: 2
     jac: [-7.318e-01 -7.332e-01 ...  0.000e+00  0.000e+00]
    nfev: 114
    njev: 2

In [181]:
# equations_flattened
# baseline.population_shell
# equations
# ff = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*xopt) for eq in equations_flattened])
# len(baseline.all_symbolic_vars)

56

In [174]:
def ode_system(t, x):
    ff = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*x) for eq in equations_flattened])
    return ff

N_shell = baseline.n_shells
R0_km = baseline.R0_km
xopt = result.x

S_opt = xopt[1:N_shell]
N_opt = xopt[N_shell+1:2*N_shell]
D_opt = xopt[2*N_shell+1:3*N_shell]
lam_opt = xopt[3*N_shell+1:4*N_shell]

c_eq = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*xopt) for eq in equations_flattened])
S_opt = c_eq[1:N_shell]
N_opt = c_eq[N_shell+1:2*N_shell]
D_opt = c_eq[2*N_shell+1:3*N_shell]

x_prop = solve_ivp(ode_system, (0, 100), xopt, method=baseline.integrator)

S_prop = x_prop[1:N_shell]
N_prop = x_prop[N_shell+1:2*N_shell]
D_prop = x_prop[2*N_shell+1:3*N_shell]
N_tot = S_prop + D_prop + N_prop
N_tot_sum = sum(N_tot,2)
S_sum = sum(S_prop,2)
D_sum = sum(D_prop,2)
N_sum = sum(N_prop,2)

R0_km[2:]-25

ValueError: operands could not be broadcast together with shapes (42,) (56,) 

In [6]:
# from scipy.optimize import show_options
# show_options('minimize', method='SLSQP')
# show_options('minimize', method='trust-constr')